In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json
from ast import literal_eval

In [6]:
c_options = Options()
c_options.add_argument("--headless=new")
driver = webdriver.Chrome(options=c_options)
#driver.quit()

In [67]:
leagues = ['EPL','La_liga','Bundesliga','Serie_A','Ligue_1']

team_links = {'EPL':[],'La_liga':[],'Bundesliga':[],'Serie_A':[],'Ligue_1':[]}

# Getting the links to each team
for league in leagues:
    driver.get(f'https://understat.com/league/{league}')

    soup = BeautifulSoup(driver.page_source)

    links_set = soup.find('table').findAll('a')

    for anchor in links_set:
        team_links[league].append(anchor.attrs["href"]) # team/Arsenal/2023


In [73]:
file = open("team_links.json",'w')
json.dump(team_links, file, indent = 4)
file.close()

In [74]:
def get_links(src, frame):
    soup = BeautifulSoup(src)
    links = []
    anchors = soup.find('div',{'id':'team-players'}).find('table').findAll('a')
    for a in anchors:
        links.append((a.attrs["href"],a.text))
    while len(frame)-1 >= len(links):
        links.append(np.nan)
    return links

In [76]:
team_frames = {'EPL':[],'La_liga':[],'Bundesliga':[],'Serie_A':[],'Ligue_1':[]}

# Getting the links to each player
for league,teams in team_links.items():
    for team in teams:
        driver.get(f'https://understat.com/{team}')
        team_frame = pd.read_html(driver.page_source)[1]
        team_frame['Player_links'] = get_links(driver.page_source, team_frame)
        team_frames[league].append(team_frame)

In [81]:
for (league,teams),(n_l,n_t) in zip(team_links.items(),team_frames.items()):
    for team,n_team in zip(teams,n_t):
        name = team.split('/')[1]
        n_team.to_csv(f"./team_data/{name}.csv", index=False)

In [103]:
team_names = []

leagues = pd.DataFrame()

# Getting all the players who play forward
for league,teams in team_links.items():
    for team,n_team in zip(teams,n_t):
        name = team.split('/')[1]
        df = pd.read_csv(f'./team_data/{name}.csv')
        df = df[:-1]
        df = df[df['Pos'].str.contains('F')]
        df['Team'] = name
        df['League'] = league
        leagues = pd.concat([leagues,df])


In [105]:
leagues.to_csv('player_data.csv', index=False)

In [3]:
df = pd.read_csv("player_data.csv")
df

,№,Player,Pos,Apps,Min,G,A,Sh90,KP90,xG,xA,xG90,xA90,Player_links,Team,League
0,1.0,Bukayo Saka,F,27.0,2311.0,13,7,3.15,2.57,12.67-0.33,8.82+1.82,0.49,0.34,"('player/7322', 'Bukayo Saka')",Arsenal,EPL
1,2.0,Kai Havertz,F M,27.0,1787.0,8,3,2.01,1.41,7.88-0.12,2.14-0.86,0.40,0.11,"('player/5220', 'Kai Havertz')",Arsenal,EPL
2,3.0,Leandro Trossard,F M,24.0,998.0,7,1,3.61,1.44,6.38-0.62,2.20+1.20,0.58,0.20,"('player/7698', 'Leandro Trossard')",Arsenal,EPL
3,6.0,Gabriel Martinelli,F,25.0,1828.0,6,4,2.41,1.87,5.81-0.19,4.55+0.55,0.29,0.22,"('player/7752', 'Gabriel Martinelli')",Arsenal,EPL
4,7.0,Eddie Nketiah,F,24.0,1015.0,5,2,3.19,0.80,4.96-0.04,1.21-0.79,0.44,0.11,"('player/6482', 'Eddie Nketiah')",Arsenal,EPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,3.0,Grejohn Kyei,F,20.0,708.0,2,0,2.92,1.02,3.79+1.79,0.49+0.49,0.48,0.06,"('player/3663', 'Grejohn Kyei')",Clermont_Foot,Ligue_1
602,6.0,Bilal Boutobba,F M,23.0,831.0,1,1,2.17,1.19,2.27+1.27,1.79+0.79,0.25,0.19,"('player/4830', 'Bilal Boutobba')",Clermont_Foot,Ligue_1
603,17.0,Komnen Andric,F,6.0,130.0,0,0,2.08,2.77,0.08+0.08,0.15+0.15,0.05,0.10,"('player/9068', 'Komnen Andric')",Clermont_Foot,Ligue_1
604,19.0,Elbasan Rashani,F M,19.0,746.0,0,0,1.21,1.21,0.85+0.85,0.56+0.56,0.10,0.07,"('player/9648', 'Elbasan Rashani')",Clermont_Foot,Ligue_1


In [ ]:
def get_data_block(soup):
    for i in range(10):
        rings = soup.findAll('script')[i].string
        if 'shotsData' in rings:
            return rings

In [ ]:
player_stats = pd.DataFrame()

def get_player_stats(row):
    link = literal_eval(row['Player_links'])[0]
    print(literal_eval(row['Player_links'])[1])
    res = requests.get(f'https://understat.com/{link}').text
    soup = BeautifulSoup(res, 'html5lib')
    
    strings = get_data_block(soup)
    ind_start = strings.index("('")+2 
    ind_end = strings.index("')") 
    json_data = strings[ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')
    global player_stats
    player_stats = pd.concat([player_stats, pd.read_json(json_data)])

In [ ]:
# Getting each player's shot history
df.apply(get_player_stats, axis=1);
player_stats.to_csv('player_stats.csv', index=False)

In [4]:
df = pd.read_csv('player_stats.csv')
df

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,season,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
0,321238,25,MissedShots,0.776,0.413,0.030224,Bukayo Saka,h,7322,OpenPlay,2019,RightFoot,11702,Arsenal,Aston Villa,3,2,2019-09-22 15:30:00,Nicolas Pepe,Pass
1,323714,43,SavedShot,0.855,0.614,0.100162,Bukayo Saka,a,7322,OpenPlay,2019,LeftFoot,11712,Manchester United,Arsenal,1,1,2019-09-30 19:00:00,NaN,None
2,323721,59,BlockedShot,0.899,0.487,0.446465,Bukayo Saka,a,7322,OpenPlay,2019,RightFoot,11712,Manchester United,Arsenal,1,1,2019-09-30 19:00:00,Calum Chambers,Pass
3,324736,22,BlockedShot,0.866,0.674,0.068281,Bukayo Saka,h,7322,OpenPlay,2019,RightFoot,11722,Arsenal,Bournemouth,1,0,2019-10-06 14:00:00,Pierre-Emerick Aubameyang,Pass
4,351606,78,SavedShot,0.864,0.340,0.109344,Bukayo Saka,h,7322,OpenPlay,2019,RightFoot,11844,Arsenal,Manchester United,2,0,2020-01-01 20:00:00,Mesut Özil,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111026,507444,93,ShotOnPost,0.949,0.514,0.658116,Alan Virginius,a,11024,OpenPlay,2022,LeftFoot,19866,Rennes,Lille,1,3,2023-02-04 20:00:00,NaN,Rebound
111027,508828,90,SavedShot,0.921,0.350,0.275698,Alan Virginius,h,11024,OpenPlay,2022,RightFoot,19872,Lille,Strasbourg,2,0,2023-02-12 14:00:00,Rémy Cabella,Pass
111028,565691,64,SavedShot,0.867,0.582,0.307400,Alan Virginius,a,11024,OpenPlay,2023,RightFoot,23573,Nice,Clermont Foot,0,0,2024-02-25 14:00:00,Habib Keita,Pass
111029,569875,20,SavedShot,0.815,0.687,0.053035,Alan Virginius,h,11024,OpenPlay,2023,LeftFoot,23599,Clermont Foot,Le Havre,2,1,2024-03-17 14:00:00,Muhammed-Cham Saracevic,Pass
